In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from nets import *
from cfgs import *
from data import *
from clip_ops.clip_ops import *
from trainer import *


In [ ]:
%matplotlib inline
save_plot = False
plt.rcParams.update({'font.size': 10, 'axes.labelsize': 'x-large'})
D = 101  # 解像度を少し下げる（3次元なので）

cfg = additive_1x3_constrained_config.cfg

# 財1, 財2の値範囲 [0, 1]
x = np.linspace(0, 1.0, D)
# 財3の値範囲 [0, c] - いくつかの固定値でスライスを表示
v3_values = [0.0, cfg.c / 4.0, cfg.c / 2.0, 3.0 * cfg.c / 4.0, cfg.c]  # v3の固定値

# v1, v2のグリッドを作成
X_tst_list = []
for v3 in v3_values:
    v1_grid, v2_grid = np.meshgrid(x, x)
    v3_grid = np.full_like(v1_grid, v3)
    X_tst = np.stack([v1_grid.flatten(), v2_grid.flatten(), v3_grid.flatten()], axis=-1)
    X_tst = np.expand_dims(X_tst, 1)  # [D*D, 1, 3]
    X_tst_list.append(X_tst)

print(f"X_tst shape for each v3: {X_tst_list[0].shape}")
print(f"v3 values: {v3_values}")

cfg.test.num_misreports = 1
cfg.test.gd_iter = 0
cfg.test.batch_size = D * D
cfg.test.num_batches = 1
cfg.test.save_output = True


In [ ]:
Net = constrained_additive_net.Net
Generator = constrained_3item_generator.Generator
clip_op_lambda = (lambda x: clip_op_constrained_3item(x, c=cfg.c))
Trainer = constrained_trainer.Trainer


In [ ]:
net = Net(cfg)
m = Trainer(cfg, "test", net, clip_op_lambda)

# 各v3の値についてテストを実行
alloc_list = []
pay_list = []
for i, X_tst in enumerate(X_tst_list):
    print(f"\nProcessing v3 = {v3_values[i]:.3f}...")
    generator = Generator(cfg, 'test', X_tst)
    m.test(generator)
    
    # 結果を読み込み
    alloc = np.load(os.path.join(cfg.dir_name, "alloc_tst_" + str(cfg.test.restore_iter) + ".npy")).reshape(D, D, 3)
    pay = np.load(os.path.join(cfg.dir_name, "pay_tst_" + str(cfg.test.restore_iter) + ".npy")).reshape(D, D, 1)
    alloc_list.append(alloc)
    pay_list.append(pay)


In [ ]:
# 財1の配分確率を可視化（各v3の値でスライス）
fig, axes = plt.subplots(1, len(v3_values), figsize=(5*len(v3_values), 4))
if len(v3_values) == 1:
    axes = [axes]

for idx, (v3, alloc) in enumerate(zip(v3_values, alloc_list)):
    ax = axes[idx]
    img = ax.imshow(alloc[::-1, :, 0], extent=[0, 1, 0, 1], vmin=0.0, vmax=1.0, cmap='YlOrRd')
    ax.set_xlabel(r'$v_1$')
    ax.set_ylabel(r'$v_2$')
    ax.set_title(f'Item 1 alloc. prob. (v3={v3:.2f})')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'alloc1_slices.pdf'), bbox_inches='tight', pad_inches=0.05)
plt.show()


In [ ]:
# 財2の配分確率を可視化（各v3の値でスライス）
fig, axes = plt.subplots(1, len(v3_values), figsize=(5*len(v3_values), 4))
if len(v3_values) == 1:
    axes = [axes]

for idx, (v3, alloc) in enumerate(zip(v3_values, alloc_list)):
    ax = axes[idx]
    img = ax.imshow(alloc[::-1, :, 1], extent=[0, 1, 0, 1], vmin=0.0, vmax=1.0, cmap='YlOrRd')
    ax.set_xlabel(r'$v_1$')
    ax.set_ylabel(r'$v_2$')
    ax.set_title(f'Item 2 alloc. prob. (v3={v3:.2f})')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'alloc2_slices.pdf'), bbox_inches='tight', pad_inches=0.05)
plt.show()


In [ ]:
# 財3の配分確率を可視化（各v3の値でスライス）
fig, axes = plt.subplots(1, len(v3_values), figsize=(5*len(v3_values), 4))
if len(v3_values) == 1:
    axes = [axes]

for idx, (v3, alloc) in enumerate(zip(v3_values, alloc_list)):
    ax = axes[idx]
    img = ax.imshow(alloc[::-1, :, 2], extent=[0, 1, 0, 1], vmin=0.0, vmax=1.0, cmap='YlOrRd')
    ax.set_xlabel(r'$v_1$')
    ax.set_ylabel(r'$v_2$')
    ax.set_title(f'Item 3 alloc. prob. (v3={v3:.2f})')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'alloc3_slices.pdf'), bbox_inches='tight', pad_inches=0.05)
plt.show()


In [ ]:
# 制約違反の可視化（財3の配分確率が制約を満たしているか）
# 下界: max(0, alloc1 + alloc2) <= alloc3
# 上界: alloc3 <= min(alloc1, alloc2)
fig, axes = plt.subplots(2, len(v3_values), figsize=(5*len(v3_values), 8))
if len(v3_values) == 1:
    axes = axes.reshape(2, 1)

for idx, (v3, alloc) in enumerate(zip(v3_values, alloc_list)):
    alloc1 = alloc[:, :, 0]
    alloc2 = alloc[:, :, 1]
    alloc3 = alloc[:, :, 2]
    
    # 下界違反: max(0, alloc1 + alloc2) - alloc3
    lower_bound = np.maximum(0, alloc1 + alloc2)
    lower_violation = np.maximum(0, lower_bound - alloc3)
    
    # 上界違反: alloc3 - min(alloc1, alloc2)
    upper_bound = np.minimum(alloc1, alloc2)
    upper_violation = np.maximum(0, alloc3 - upper_bound)
    
    # 下界違反の可視化
    ax = axes[0, idx]
    img = ax.imshow(lower_violation[::-1, :], extent=[0, 1, 0, 1], vmin=0.0, cmap='Reds')
    ax.set_xlabel(r'$v_1$')
    ax.set_ylabel(r'$v_2$')
    ax.set_title(f'Lower bound violation (v3={v3:.2f})')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)
    
    # 上界違反の可視化
    ax = axes[1, idx]
    img = ax.imshow(upper_violation[::-1, :], extent=[0, 1, 0, 1], vmin=0.0, cmap='Reds')
    ax.set_xlabel(r'$v_1$')
    ax.set_ylabel(r'$v_2$')
    ax.set_title(f'Upper bound violation (v3={v3:.2f})')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'constraint_violations.pdf'), bbox_inches='tight', pad_inches=0.05)
plt.show()
